In [1]:
from ultralytics import YOLO
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import shutil
from glob import glob

In [2]:
base_path = '../../dataset/sagittal'
train_image = os.path.join(base_path, 'images/train')
train_label = os.path.join(base_path, 'labels/train')
test_image = os.path.join(base_path, 'images/test')
test_label = os.path.join(base_path, 'labels/test')
val_image = os.path.join(base_path, 'images/val')
val_label = os.path.join(base_path, 'labels/val')

In [3]:
# seed 및 GPU 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# Read and sort the file list to ensure consistent order
file_list = sorted(os.listdir(train_image))
random_file = random.choice(file_list).split('.')[0]
print("Image :", random_file)

# Read labels from the file
with open(os.path.join(train_label, f'{random_file}.txt'), 'r') as f:
    labels = f.readlines()
    labels = labels[0].split(' ')

# Read and process the image
img = cv2.imread(os.path.join(train_image, f'{random_file}.jpg'), cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Image : 00085_156


In [4]:
# 파일 개수 세는 함수
def count_files(directory):
    return len(os.listdir(directory))

num_train_images = count_files(train_image)
num_test_images = count_files(test_image)
num_train_labels = count_files(train_label)
num_test_labels = count_files(test_label)
print(f'Train images: {num_train_images}, Train labels: {num_train_labels}')
print(f'Test images: {num_test_images}, Test labels: {num_test_labels}\n')

Train images: 264, Train labels: 264
Test images: 70, Test labels: 70



In [5]:
#실행XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

# val_image = os.path.join(base_path, 'images/val')
# val_label = os.path.join(base_path, 'labels/val')

# # Get list of train images
# train_images = os.listdir(train_image)

# # Calculate the number of validation images
# num_train_images = len(train_images)
# val_split = int(num_train_images * 0.2)

# # Sample validation images from train images
# random.seed(42)
# val_images = random.sample(train_images, val_split)

# # Remove existing val directories if they exist
# if os.path.exists(val_image):
#     shutil.rmtree(val_image)
# if os.path.exists(val_label):
#     shutil.rmtree(val_label)

# # Create new val directories
# os.makedirs(val_image, exist_ok=True)
# os.makedirs(val_label, exist_ok=True)

# # Move selected images and corresponding labels to val directories
# for image in val_images:
#     shutil.move(os.path.join(train_image, image), val_image)
#     label = os.path.splitext(image)[0] + '.txt'
#     shutil.move(os.path.join(train_label, label), val_label)

# # Debugging: Print the count of files in each directory
# print("Train images:", len(os.listdir(train_image)))
# print("Validation images:", len(os.listdir(val_image)))
# print("Train labels:", len(os.listdir(train_label)))
# print("Validation labels:", len(os.listdir(val_label)))

Train images: 212
Validation images: 52
Train labels: 212
Validation labels: 52


In [6]:
#v8n
experiment_name = 'train'
model_size = 'v8n' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_n = YOLO('yolov8n.pt') #이름수정주의 2개
results_n = model_n.train( #이름수정주의 2개
    data='sagittal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 6.23M/6.23M [00:00<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8n, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /home/under1/Detect/dataset/sagittal/labels/train... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<00:00, 755.93it/s]

train: New cache created: /home/under1/Detect/dataset/sagittal/labels/train.cache



val: Scanning /home/under1/Detect/dataset/sagittal/labels/val... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<00:00, 859.79it/s]

...

    200/200      2.14G     0.4093     0.2827     0.8281          4        640: 100%|██████████| 14/14 [00:01<00:00, 12.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 11.91it/s]

                   all         52         54      0.588      0.733      0.655      0.506



200 epochs completed in 0.106 hours.
Optimizer stripped from runs/train/train_v8n/weights/last.pt, 6.3MB
Optimizer stripped from runs/train/train_v8n/weights/best.pt, 6.3MB

Validating runs/train/train_v8n/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.12it/s]


                   all         52         54      0.598      0.739      0.738      0.551
              negative         52         30      0.666      0.867      0.831      0.652
              positive         52         24       0.53      0.611      0.646       0.45
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/train/train_v8n


In [7]:
result_n = model_n.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.68it/s]


                   all         52         54      0.596      0.739      0.737      0.557
              negative         52         30      0.663      0.867      0.828      0.657
              positive         52         24       0.53      0.611      0.646      0.457
Speed: 0.2ms preprocess, 11.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/train_v8n


In [8]:
#v8s
experiment_name = 'train'
model_size = 'v8s' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_s = YOLO('yolov8s.pt') #이름수정주의 2개
results_s = model_s.train( #이름수정주의 2개
    data='sagittal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 21.5M/21.5M [00:01<00:00, 11.3MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8s, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8s/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8s
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/14 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      4.23G      2.036      6.521      1.711         11        640: 100%|██████████| 14/14 [00:02<00:00,  6.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.69it/s]

...

    200/200      4.18G     0.3823     0.2539     0.8191          4        640: 100%|██████████| 14/14 [00:01<00:00,  9.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.31it/s]

                   all         52         54      0.652      0.702      0.696      0.547



200 epochs completed in 0.143 hours.
Optimizer stripped from runs/train/train_v8s/weights/last.pt, 22.5MB
Optimizer stripped from runs/train/train_v8s/weights/best.pt, 22.5MB

Validating runs/train/train_v8s/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.43it/s]


                   all         52         54      0.754      0.617      0.722      0.564
              negative         52         30      0.896      0.577      0.785      0.627
              positive         52         24      0.612      0.658      0.658      0.501
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/train/train_v8s


In [9]:
result_s = model_s.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.36it/s]


                   all         52         54      0.755      0.618      0.723      0.566
              negative         52         30      0.896      0.574      0.786      0.629
              positive         52         24      0.614      0.662      0.659      0.503
Speed: 0.3ms preprocess, 9.5ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/train_v8s


In [10]:
#v8m
experiment_name = 'train'
model_size = 'v8m' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_m = YOLO('yolov8m.pt') #이름수정주의 2개
results_m = model_m.train( #이름수정주의 2개
    data='sagittal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 49.7M/49.7M [00:04<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8m, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8m/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8m
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/14 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200       7.1G      1.858      5.524      1.567         29        640:  93%|█████████▎| 13/14 [00:02<00:00,  4.87it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backw

...

    200/200      7.11G     0.3083     0.2435     0.8224          4        640: 100%|██████████| 14/14 [00:02<00:00,  5.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.98it/s]

                   all         52         54      0.634      0.673      0.584      0.426



200 epochs completed in 0.246 hours.
Optimizer stripped from runs/train/train_v8m/weights/last.pt, 52.1MB
Optimizer stripped from runs/train/train_v8m/weights/best.pt, 52.1MB

Validating runs/train/train_v8m/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.05it/s]


                   all         52         54       0.63      0.679      0.692      0.519
              negative         52         30       0.79      0.567      0.765      0.608
              positive         52         24      0.469      0.792      0.618       0.43
Speed: 0.1ms preprocess, 3.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/train/train_v8m


In [11]:
result_m = model_m.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  75%|███████▌  | 3/4 [00:01<00:00,  2.54it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.49it/s]


                   all         52         54       0.63      0.679      0.692      0.519
              negative         52         30      0.791      0.567      0.765      0.608
              positive         52         24       0.47      0.792      0.618       0.43
Speed: 0.2ms preprocess, 15.4ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/train_v8m


In [12]:
#v8l
experiment_name = 'train'
model_size = 'v8l' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_l = YOLO('yolov8l.pt') #이름수정주의 2개
results_l = model_l.train( #이름수정주의 2개
    data='sagittal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 83.7M/83.7M [00:07<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8l, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8l/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8l
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/14 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      11.5G      1.768      6.001      1.682         11        640: 100%|██████████| 14/14 [00:05<00:00,  2.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.27it/s]

...

    200/200      11.5G     0.3582     0.2702     0.8665          4        640: 100%|██████████| 14/14 [00:03<00:00,  3.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.94it/s]

                   all         52         54      0.591        0.7      0.627      0.485



200 epochs completed in 0.366 hours.
Optimizer stripped from runs/train/train_v8l/weights/last.pt, 87.7MB
Optimizer stripped from runs/train/train_v8l/weights/best.pt, 87.7MB

Validating runs/train/train_v8l/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 43608150 parameters, 0 gradients, 164.8 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.49it/s]


                   all         52         54      0.554       0.76      0.673       0.51
              negative         52         30      0.696      0.686      0.731      0.547
              positive         52         24      0.412      0.833      0.615      0.473
Speed: 0.1ms preprocess, 5.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/train/train_v8l


In [13]:
result_l = model_l.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 43608150 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.22s/it]


                   all         52         54      0.554       0.76      0.672      0.511
              negative         52         30      0.696      0.686      0.729      0.545
              positive         52         24      0.412      0.833      0.616      0.477
Speed: 0.2ms preprocess, 78.2ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/train_v8l


In [14]:
#v8x
experiment_name = 'train'
model_size = 'v8x' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{model_size}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_x = YOLO('yolov8x.pt') #이름수정주의 2개
results_x = model_x.train( #이름수정주의 2개
    data='sagittal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,
    project='runs',   
    name=os.path.join(experiment_name, f'{experiment_name}_{model_size}'),  
    exist_ok=True,
)

def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")

100%|██████████| 131M/131M [00:11<00:00, 11.7MB/s] 


New https://pypi.org/project/ultralytics/8.2.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=sagittal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=train_v8x, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /home/under1/Detect/dataset/sagittal/labels/train.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/train/train_v8x/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/train/train_v8x
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      15.2G      1.674      6.414      1.666         29        640:  93%|█████████▎| 13/14 [00:05<00:00,  2.24it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
      1/200      15.3G      1.684      6.876      1.667         11        640: 100%|██████████| 14/14 [00:06<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.97it/s]

...

    200/200      15.2G     0.3647     0.2692     0.9425          4        640: 100%|██████████| 14/14 [00:05<00:00,  2.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.51it/s]

                   all         52         54      0.544      0.613      0.566      0.403



200 epochs completed in 0.557 hours.
Optimizer stripped from runs/train/train_v8x/weights/last.pt, 136.8MB
Optimizer stripped from runs/train/train_v8x/weights/best.pt, 136.7MB

Validating runs/train/train_v8x/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.26it/s]


                   all         52         54      0.603      0.708      0.656      0.501
              negative         52         30      0.757      0.667      0.727      0.579
              positive         52         24      0.449       0.75      0.586      0.423
Speed: 0.1ms preprocess, 8.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/train/train_v8x


In [15]:
result_x = model_x.val()

Ultralytics YOLOv8.2.16 🚀 Python-3.8.19 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /home/under1/Detect/dataset/sagittal/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4 [00:00<?, ?it/s]/home/under1/.conda/envs/detect/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 1/4 [00:06<00:20,  6.79s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:22<00:00,  5.66s/it]


                   all         52         54      0.594      0.667      0.627      0.479
              negative         52         30      0.756      0.667      0.732      0.579
              positive         52         24      0.431      0.667      0.521      0.379
Speed: 0.2ms preprocess, 347.2ms inference, 0.0ms loss, 66.0ms postprocess per image
Results saved to runs/train_v8x
